Berikut adalah data tentang class Gaji seseorang. Data ini memilki beberapa column predictor dan 1 column output untuk 2 class label, Anda di minta untuk membuat model untuk memprediksi gaji seseorang, dengan ketentuan sebagai berikut:

0 : Gaji 6 juta

Silahkan download file train.csv sebagai data training, dan gunakan data test.csv sebagai data test yang akan di jadikan penilaian di project ini .

Sanbercode Data Science Bootcamp

Evaluation
Evaluasi matriks yang di gunakan adalah AUC, jadi pastikan anda melakukan scoring metrics roc_auc terhadap model yang anda buat.

Anda dapat mensubmit hasil sebanyak 5 kali dalam sehari.

Terimakasih.

Happy Coding!!

Data Description

Terdapat tiga file yang digunakan pada final project ini, file train.csv berisi dataset yang memiliki kolom label digunakan untuk traning model, sedangkan file test.csv berisi dataset tanpa kolom label, yang nantinya teman-teman dapat menambahkan kolom 'Gaji' berisi prediksi teman-teman sesuai ketentuan pada tab Overview. Kemudian teman-teman diminta untuk membuat dataset baru berisi kolom 'id' dan kolom 'Gaji' yang diambil dari dataset test yang telah teman-teman tambahkan prediksinya, dataset baru ini yang nantinya akan dijadikan file submission. Contoh isi dari submission dapat teman-teman lihat pada file sampleforsubmission.csv

File descriptions

train.csv - file training set test.csv - file test set sampleforsubmission.csv - contoh file submission dalam format yang benar

Data fields

id - id unique yang dimiliki setiap sample Umur - umur yang dimiliki setiap sample Kelas Pekerja - kelompok kelas pekerjaan masing-masing sample Berat Akhir - berisi nilai akumulasi berdasarkan populasi, ras, dan gender dengan umur 16+ suatu wilayah, sample yang diambil dari wilayah yang memiliki karakteristik demografis yang sama akan memiliki nilai berat akhir yang sama Pendidikan - tingkat pendidikan terakhir masing-masing sample Jmlh Tahun Pendidikan - berisi jumlah tahun masing-masing sample mengenyam pendidikan Status Perkawinan - status perkawinan masing-masing sample Pekerjaan - pekerjaan saat ini masing-masing sample Jenis Kelamin - jenis kelamin masing-masing sample Keuntungan Kapital - keuntungan yang didapat jika sample menjual semua aset miliknya Kerugian Kapital - kerugian yang didapat jika sample menjual semua aset miliknya Jam per Minggu - jam kerja masing-masing sample setiap minggunya Gaji - nilai gaji masing-masing sample apakah kurang dari sama dengan 6 juta atau lebih dari 6 juta

In [223]:
# Import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import normalize, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [224]:
# Read files and save to data frame 
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_example = pd.read_csv('sampleforsubmission.csv')

In [225]:
# Display test data 
df_test

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu
0,35994,30,Pemerintah Lokal,99761,D4,11,Cerai,Pemuka Agama,Perempuan,0.0,0.0,40.0
1,35995,28,Wiraswasta,202206,SMA,9,Menikah,Mesin Inspeksi,Laki2,0.0,0.0,50.0
2,35996,34,Pemerintah Provinsi,199934,Pendidikan Tinggi,10,Menikah,Pemuka Agama,Laki2,0.0,0.0,40.0
3,35997,51,Wiraswasta,373448,Pendidikan Tinggi,10,Menikah,Ekesekutif Managerial,Laki2,0.0,28028000.0,40.0
4,35998,20,?,144685,Pendidikan Tinggi,10,Belum Pernah Menikah,?,Perempuan,0.0,22428000.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9594,45588,58,Pemerintah Negara,159021,9th,5,Menikah,Petani,Laki2,0.0,0.0,40.0
9595,45589,45,Pekerja Bebas Bukan Perusahan,144086,11th,7,Menikah,Perbaikan Kerajinan,Laki2,0.0,0.0,50.0
9596,45590,20,Wiraswasta,238685,Pendidikan Tinggi,10,Belum Pernah Menikah,Servis Lainnya,Perempuan,0.0,0.0,32.0
9597,45591,41,Pemerintah Lokal,249039,SMA,9,Berpisah,Pemuka Agama,Perempuan,0.0,0.0,40.0


In [226]:
# Display train data 
df_train

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,21,Wiraswasta,242912,SMA,9,Belum Pernah Menikah,Servis Lainnya,Perempuan,0,0,35,<= 6 juta
1,1,49,Wiraswasta,140782,10th,6,Cerai,Ekesekutif Managerial,Perempuan,0,0,40,<= 6 juta
2,2,44,Wiraswasta,120057,D3,12,Menikah,Ekesekutif Managerial,Laki2,61404000,0,45,> 6 juta
3,3,24,Wiraswasta,194630,Sarjana,13,Belum Pernah Menikah,Spesialis,Laki2,0,0,35,<= 6 juta
4,4,33,Wiraswasta,219619,Master,14,Menikah,Spesialis,Laki2,210336000,0,40,> 6 juta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35989,35989,47,Pekerja Bebas Bukan Perusahan,148169,SMA,9,Menikah,Perbaikan Kerajinan,Laki2,0,0,40,<= 6 juta
35990,35990,69,Pekerja Bebas Perusahaan,264722,D3,12,Menikah,Sales,Laki2,0,0,40,> 6 juta
35991,35991,24,Pekerja Bebas Bukan Perusahan,31606,Sarjana,13,Menikah,Spesialis,Perempuan,0,0,20,> 6 juta
35992,35992,47,Wiraswasta,197836,SMA,9,Menikah,Sales,Laki2,0,0,45,<= 6 juta


In [227]:
# Display example for submission 
df_example.head()

,id,Gaji
0,35994,0
1,35995,0
2,35996,0
3,35997,1
4,35998,0


In [228]:
# # Initialization

# df_test = df_test.drop(['Berat Akhir','Keuntungan Kapital','Kerugian Capital'], axis = 1)
# X = df_train.drop(['Gaji','Berat Akhir','Keuntungan Kapital','Kerugian Capital'], axis = 1)

X = df_train.drop('Gaji', axis = 1)
y = df_train['Gaji']
ID = df_test['id']

In [229]:
# pre-processing part 1: Filling up missing values 

# Replace ? with modus value 'Wiraswasta'
X['Kelas Pekerja'] = X['Kelas Pekerja'].replace({'?': 'Wiraswasta'})
X['Pekerjaan'] = X['Pekerjaan'].replace({'?': 'Spesialis'})

# Replace ? with modus value 'Spesialis'
df_test['Kelas Pekerja'] = df_train['Kelas Pekerja'].replace({'?': 'Wiraswasta'})
df_test['Pekerjaan'] = df_train['Pekerjaan'].replace({'?': 'Spesialis'})

In [230]:
# Pre-processing part 2: Encoding

# Encoding feature data: Ordinal data
obj_list = {
    'SD': 0,
    'SMA' : 2,                    
    'Pendidikan Tinggi' : 5,        
    'Sarjana' : 5,                  
    'Master' : 6,                   
    'D4' : 4,                       
    '11th' : 2,                     
    'D3' : 3,                       
    '10th' : 2,                     
    '7th-8th' : 1,                  
    'Sekolah Professional' : 2,     
    '9th' : 1,                      
    '12th' : 2,                     
    'Doktor' : 7,                   
    '5th-6th' : 0,                  
    '1st-4th' : 0,                  
    'SD' : 0}       

X['Pendidikan'] = X['Pendidikan'].replace(obj_list)
df_test['Pendidikan'] = df_test['Pendidikan'].replace(obj_list)

# Encoding feature data: Nonordinal data
X = pd.get_dummies(X)
df_test = pd.get_dummies(df_test)

# Encoding target data
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [231]:
# Pre-processing part 3: Normalization
X = normalize(X) 
df_test = normalize(df_test) 

In [232]:
# Tune hyperparameter

# Split training data and valdation data 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=21)

model_knn = KNeighborsClassifier()
parameter_knn = {'n_neighbors': np.arange(15,26), 'weights':['uniform','distance']}

gscv_knn = GridSearchCV(model_knn, param_grid = parameter_knn, scoring = 'roc_auc', cv = 5)
gscv_knn.fit(X_train, y_train)

y_pred = gscv_knn.predict(X_val)
score_knn = roc_auc_score(y_val, y_pred)

print(score_knn)
print('Kombinasi KNN adalah',gscv_knn.best_params_)
print('best score', gscv_knn.best_score_)
print('roc auc scorer:', roc_auc_score(y_val, y_pred))

0.6109259710232837
Kombinasi KNN adalah {'n_neighbors': 19, 'weights': 'distance'}
best score 0.6587720033257884
roc auc scorer: 0.6109259710232837


In [233]:
# y_pred = gscv_knn.predict(X_val)
# score_knn = roc_auc_score(y_val, y_pred)

# print(score_knn)
# print('Kombinasi KNN adalah',gscv_knn.best_params_)
# print('best score', gscv_knn.best_score_)
# print('roc auc scorer:', roc_auc_score(y_val, y_pred))

In [234]:
model_tree = DecisionTreeClassifier()
parameter_tree = {'criterion': ['gini','entropy'], 'splitter':['best','random']}

gscv_tree = GridSearchCV(model_tree, param_grid = parameter_tree, scoring = 'roc_auc', cv = 5)
gscv_tree.fit(X_train, y_train)

y_pred = gscv_tree.predict(X_val)
score_tree = roc_auc_score(y_val, y_pred)

# print(score_tree)

print('Kombinasi Decision Tree',gscv_tree.best_params_)
print('best score', gscv_tree.best_score_)
print('roc auc scorer:', roc_auc_score(y_val, y_pred))

Kombinasi Decision Tree {'criterion': 'entropy', 'splitter': 'random'}
best score 0.7259569766725456
roc auc scorer: 0.7236657141110898


In [235]:
model_forest = RandomForestClassifier()
parameter_forest = {'n_estimators': np.arange(5,10), 'max_depth':np.arange(2,22)}

gscv_forest = GridSearchCV(model_forest, param_grid = parameter_forest, scoring = 'roc_auc', cv = 5)
gscv_forest.fit(X_train, y_train)

y_pred = gscv_forest.predict(X_val)
score_forest = roc_auc_score(y_val, y_pred)

# print(score_forest)

print('Kombinasi Random Forest',gscv_forest.best_params_)
print('best score', gscv_forest.best_score_)
print('roc auc scorer:', roc_auc_score(y_val, y_pred))

Kombinasi Random Forest {'max_depth': 13, 'n_estimators': 9}
best score 0.8700831255839688
roc auc scorer: 0.7027081362394341


In [236]:
model_knn = KNeighborsClassifier(n_neighbors = 16, weights = 'distance')
model_knn.fit(X_train, y_train)

y_pred = model_knn.predict(X_val)

cv_score = cross_validate(model_knn, X, y, cv = 10, return_train_score = True, scoring = 'roc_auc')
test_score = cv_score['test_score'].mean()
train_score = cv_score['train_score'].mean()

print(confusion_matrix(y_val, y_pred))

report = classification_report(y_val, y_pred)
print(report)

print(roc_auc_score(y_val, y_pred))

[[6618  224]
 [1592  565]]
              precision    recall  f1-score   support

           0       0.81      0.97      0.88      6842
           1       0.72      0.26      0.38      2157

    accuracy                           0.80      8999
   macro avg       0.76      0.61      0.63      8999
weighted avg       0.78      0.80      0.76      8999

0.6145994557328628


In [237]:
model_tree = DecisionTreeClassifier(criterion = 'entropy', splitter = 'best')
model_tree.fit(X_train, y_train)

y_pred = model_tree.predict(X_val)

cv_score = cross_validate(model_tree, X, y, cv = 10, return_train_score = True, scoring = 'roc_auc')
test_score = cv_score['test_score'].mean()
train_score = cv_score['train_score'].mean()

print(confusion_matrix(y_val, y_pred))

report = classification_report(y_val, y_pred)
print(report)

print(roc_auc_score(y_val, y_pred))

[[5846  996]
 [ 951 1206]]
              precision    recall  f1-score   support

           0       0.86      0.85      0.86      6842
           1       0.55      0.56      0.55      2157

    accuracy                           0.78      8999
   macro avg       0.70      0.71      0.71      8999
weighted avg       0.79      0.78      0.78      8999

0.7067692022479173


In [238]:
model_forest = RandomForestClassifier(max_depth = 12, n_estimators = 8)
model_forest.fit(X_train, y_train)

y_pred = model_forest.predict(X_val)

cv_score = cross_validate(model_forest, X, y, cv = 10, return_train_score = True, scoring = 'roc_auc')
test_score = cv_score['test_score'].mean()
train_score = cv_score['train_score'].mean()

print(confusion_matrix(y_val, y_pred))

report = classification_report(y_val, y_pred)
print(report)

print(roc_auc_score(y_val, y_pred))

[[6440  402]
 [1185  972]]
              precision    recall  f1-score   support

           0       0.84      0.94      0.89      6842
           1       0.71      0.45      0.55      2157

    accuracy                           0.82      8999
   macro avg       0.78      0.70      0.72      8999
weighted avg       0.81      0.82      0.81      8999

0.6959355595948935


In [242]:
# Export to csv file
# df_test.drop(['Berat Akhir','Keuntungan Kapital','Kerugian Capital'], axis = 1)
# y_final = gscv_forest.predict(df_test)
y_final = gscv_knn.predict(df_test)

result = pd.DataFrame({'id': ID, 'Gaji': y_final})
result.to_csv('Gaji5.csv', index = False)
# df_test